In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L2\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['COAST.csv', 'EAST.csv', 'FWEST.csv', 'NCENT.csv', 'NORTH.csv', 'SCENT.csv', 'SOUTH.csv', 'WEST.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[1]))

,Net,Temprature
0,1350.784678,4.6
1,1333.434943,4.6
2,1323.908498,4.5
3,1325.951405,4.3
4,1339.675290,3.9
...,...,...
43818,1417.306742,13.4
43819,1411.761157,13.8
43820,1403.990584,13.7
43821,1387.051521,12.5


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[1]

zoneData = pd.read_csv(os.path.join(path, fileName))
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_EAST,Temp_EAST
0,1350.784678,4.6
1,1350.784678,4.6
2,1333.434943,4.6
3,1323.908498,4.5
4,1325.951405,4.3
...,...,...
43819,1417.306742,13.4
43820,1411.761157,13.8
43821,1403.990584,13.7
43822,1387.051521,12.5


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_EAST,Temp_EAST,Lag-2,Lag-7
0,1811.886000,4.6,1566.078230,1350.784678
1,1795.960738,3.0,1563.801029,1350.784678
2,1796.887029,1.6,1550.922231,1333.434943
3,1832.138977,0.2,1559.483464,1323.908498
4,1806.797437,-1.3,1563.313440,1325.951405
...,...,...,...,...
43651,1417.306742,13.4,1360.791751,1222.397590
43652,1411.761157,13.8,1388.562537,1237.349389
43653,1403.990584,13.7,1387.373894,1236.197626
43654,1387.051521,12.5,1350.918789,1216.187797


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 13s 8ms/step - loss: 0.0127 - mape: 7964.6104 - mae: 0.0854 - val_loss: 0.0091 - val_mape: 24.6471 - val_mae: 0.0744
Epoch 2/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0067 - mape: 6550.4370 - mae: 0.0630 - val_loss: 0.0192 - val_mape: 32.0672 - val_mae: 0.1031
Epoch 3/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0047 - mape: 5490.2021 - mae: 0.0526 - val_loss: 0.0233 - val_mape: 39.4225 - val_mae: 0.1167
Epoch 4/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0043 - mape: 5023.5938 - mae: 0.0504 - val_loss: 0.0175 - val_mape: 31.7451 - val_mae: 0.1011
Epoch 5/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0039 - mape: 4891.3677 - mae: 0.0481 - val_loss: 0.0244 - val_mape: 37.6646 - val_mae: 0.1178
Epoch 6/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0036 - mape: 4937.8115 - mae: 0.0463 - val_loss: 0.0195

1105/1105 [==============================] - 8s 7ms/step - loss: 0.0015 - mape: 1019.5509 - mae: 0.0299 - val_loss: 0.0219 - val_mape: 30.9027 - val_mae: 0.1081
Epoch 49/100
1105/1105 [==============================] - 7s 7ms/step - loss: 0.0015 - mape: 1210.6473 - mae: 0.0296 - val_loss: 0.0215 - val_mape: 30.4734 - val_mae: 0.1063
Epoch 50/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0015 - mape: 1121.4833 - mae: 0.0294 - val_loss: 0.0219 - val_mape: 30.9318 - val_mae: 0.1091
Epoch 51/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0015 - mape: 857.8231 - mae: 0.0294 - val_loss: 0.0165 - val_mape: 26.6040 - val_mae: 0.0933
Epoch 52/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0014 - mape: 1259.7509 - mae: 0.0292 - val_loss: 0.0190 - val_mape: 29.2487 - val_mae: 0.1019
Epoch 53/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0015 - mape: 1297.6444 - mae: 0.0293 - val_loss: 0.0233 - val_ma

1105/1105 [==============================] - 9s 8ms/step - loss: 9.2482e-04 - mape: 761.3038 - mae: 0.0233 - val_loss: 0.0215 - val_mape: 30.7367 - val_mae: 0.1078
Epoch 96/100
1105/1105 [==============================] - 9s 8ms/step - loss: 9.5453e-04 - mape: 579.0182 - mae: 0.0236 - val_loss: 0.0221 - val_mape: 30.7053 - val_mae: 0.1080
Epoch 97/100
1105/1105 [==============================] - 9s 8ms/step - loss: 9.0526e-04 - mape: 720.6031 - mae: 0.0230 - val_loss: 0.0221 - val_mape: 31.6259 - val_mae: 0.1094
Epoch 98/100
1105/1105 [==============================] - 8s 8ms/step - loss: 9.0696e-04 - mape: 835.5345 - mae: 0.0230 - val_loss: 0.0209 - val_mape: 30.2147 - val_mae: 0.1061
Epoch 99/100
1105/1105 [==============================] - 9s 8ms/step - loss: 9.0955e-04 - mape: 670.6819 - mae: 0.0230 - val_loss: 0.0225 - val_mape: 31.3141 - val_mae: 0.1101
Epoch 100/100
1105/1105 [==============================] - 9s 8ms/step - loss: 8.9280e-04 - mape: 716.5109 - mae: 0.0228 - val_l

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 792.938371181488


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 5ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.1261042 , 0.1052054 , 0.09761809, ..., 0.45604992, 0.4198601 ,
       0.39123443], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.28107855, 0.24879522, 0.22672484, ..., 0.33648852, 0.32725215,
       0.31132498])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,1302.371352,1018.155762
1,1243.165263,979.828308
2,1202.689236,965.913574
3,1208.718450,964.505981
4,1235.412147,979.738831
...,...,...
4363,1417.306742,1676.644775
4364,1411.761157,1666.770874
4365,1403.990584,1623.260498
4366,1387.051521,1556.890137


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

12.933630084700349